In [1]:
import cv2
import pickle
import numpy as np


In [3]:
gmodeldict = pickle.load(open('GenderModel.p', 'rb'))
gmodel = gmodeldict['model']



In [14]:
import dlib

model_path = "path where you stored shape_predictor_81_face_landmarks.dat"

model = dlib.shape_predictor(model_path)

detector = dlib.get_frontal_face_detector()


smodeldict = pickle.load(open('FaceShapeModel.p', 'rb'))
shapemodel = smodeldict['model']

In [15]:
import math

def distance(point1, point2):
    # Unpack the coordinates from the input arrays
    x1, y1 = point1
    x2, y2 = point2

    # Calculate the distance
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    return distance


def angle(p1, p2, p3):
    # Unpack the coordinates from the input arrays
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    
    # Calculate the vectors of the two lines
    vector1 = (x1 - x2, y1 - y2)
    vector2 = (x3 - x2, y3 - y2)
    
    # Calculate the dot product and magnitudes of the vectors
    dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]
    magnitude1 = math.sqrt(vector1[0] ** 2 + vector1[1] ** 2)
    magnitude2 = math.sqrt(vector2[0] ** 2 + vector2[1] ** 2)
    
    # Calculate the cosine of the angle between the lines
    cosine = dot_product / (magnitude1 * magnitude2)
    
    # Calculate the angle in radians and convert to degrees
    angle_rad = math.acos(cosine)
    angle_deg = math.degrees(angle_rad)
    
    return angle_deg


def midpoint(point1, point2):
    # Unpack the coordinates from the input arrays
    x1, y1 = point1
    x2, y2 = point2
    
    # Calculate the midpoint coordinates
    midpoint_x = (x1 + x2) / 2
    midpoint_y = (y1 + y2) / 2
    
    # Create an array to store the midpoint coordinates
    midpoint = [midpoint_x, midpoint_y]
    
    return midpoint



## Video input

In [27]:
from imutils import face_utils


cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()

    frame_res = cv2.resize(frame, (80, 80))

    imgp = cv2.cvtColor(frame_res, cv2.COLOR_BGR2GRAY)

    imgp = np.array(imgp)

    imgp = imgp/255

    # Add batch dimension to match the model's input shape
    input_data = np.expand_dims(imgp, axis=0)

    prediction = gmodel.predict(input_data)

    gender = prediction.argmax(axis = 1)

    
    if gender==1:
        str = 'Male'
    else:
        str = 'Female'

    cv2.putText(frame, "Gender: " + str, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1,(150,100,0), 2, cv2.LINE_AA )



    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    dets = detector(img)


    for face in dets:
        shape = model(img, face)
        coords = face_utils.shape_to_np(shape)   # our coordinates of face landmarks are stored here
        
        for (x, y) in coords:
            # Process or visualize the landmark coordinates
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)



  
    l3 = coords[3]
    l15 = coords[15]
    l70 = coords[70]
    l76 = coords[76]
    l80 = coords[80]
    l73 = coords[73]
    l9 = coords[9]
    l13 = coords[13]
    l5 = coords[5]
    l7 = coords[7]
    l11 = coords[11]
    l8 = coords[8]
    l10 = coords[10]

    d1 = distance(l3, l15)
    d2 = distance(l76, l80)
    d3 = distance(midpoint(l70, l73), l9)
    d4 = distance(l9, l13)
    d5 = distance(l5, l13)
    d6 = distance(l7, l11)
    d7 = distance(l8, l10)


    DD = d1 + d2 + d3 + d4 + d5 + d6 + d7

    D1 = d1/DD
    D2 = d2/DD
    D3 = d3/DD
    D4 = d4/DD
    D5 = d5/DD
    D6 = d6/DD
    D7 = d7/DD



    R1 = D2/D1
    R2 = D1/D3
    R3 = D2/D3
    R4 = D1/D5
    R5 = D6/D5
    R6 = D4/D6
    R7 = D6/D1
    R8 = D5/D2
    R9 = D4/D5
    R10 = D7/D6


    A1 = angle(midpoint(l70, l73), l9, l11)
    A2 = angle(midpoint(l70, l73), l9, l13)
    A3 = angle(l3, l15, l13)

    features = []

    features.append(R1)
    features.append(R2)
    features.append(R3)
    features.append(R4)
    features.append(R7)
    features.append(R8)
    features.append(R10)
    features.append(D1)
    features.append(D2)
    features.append(D3)
    features.append(D5)
    features.append(D6)
    features.append(A1)
    features.append(A2)

    features = np.array(features)
    features = features.reshape(1, -1)


    predshape = shapemodel.predict(features)

    if predshape == 0:
        str = "Heart"
    elif predshape == 1:
        str = "oblong"
    elif predshape == 2:
        str = "Oval"
    elif predshape == 3:
        str = "Round"
    else:
        str = "Square"

    cv2.putText(frame, "Face Shape:" + str, (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,0), 2, cv2.LINE_AA )

    cv2.imshow("Live Camera", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 35ms/step
